In [7]:
# import depencdencies 
import pandas as pd
import math

# extract csv file containing aleady computed latitude and longitude in decimal degrees
ndf = pd.read_csv('wpi_jusrong_distance.csv')
ndf = pd.DataFrame(ndf)
ndf.head()

,Unnamed: 0,Wpi_country_code,Main_port_name,Latitude_degrees,Latitude_minutes,Latitude_hemisphere,Longitude_degrees,Longitude_minutes,Longitude_hemisphere,latitude_lag,longitude_lag,Latitude_decimal_degree,Longitude_decimal_degree,latitude_rad,longitude_rad,distance_jurong_island,distance_jurong_island_meters
0,0,IS,KEFLAVIK,64,0,N,22,33,W,1,-1,64.000000,-22.550000,1.117011,-0.393572,1.153837e+07,1.153837e+07
1,1,IS,STRAUMSVIK,64,3,N,22,3,W,1,-1,64.050000,-22.050000,1.117883,-0.384845,1.152321e+07,1.152321e+07
2,2,IS,HAFNARFJORDUR,64,4,N,21,57,W,1,-1,64.066667,-21.950000,1.118174,-0.383100,1.152085e+07,1.152085e+07
3,3,IS,SKERJAFJORDUR,64,9,N,22,1,W,1,-1,64.150000,-22.016667,1.119629,-0.384263,1.153204e+07,1.153204e+07
4,4,IS,REYKJAVIK,64,9,N,21,56,W,1,-1,64.150000,-21.933333,1.119629,-0.382809,1.152865e+07,1.152865e+07


In [8]:
# get colunms of interest and convert it to a list.
lat_rad = ndf['Latitude_decimal_degree']
lad_list = lat_rad.values.tolist()


In [9]:
lon_deg = ndf['Longitude_decimal_degree']
lon_list = lon_deg.values.tolist()


In [10]:
# assign a variable for the latitudes and longitude of the distress call
lat_deg_dis = 32.610982
lon_deg_dis = -38.706256


In [ ]:
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(lat_a, lon_a, lat_series, lon_series):
    # Earth's radius in kilometers
    radius = 6371.0

    # Convert point A's coordinates from degrees to radians
    lat_a_rad = radians(lat_a)
    lon_a_rad = radians(lon_a)

    distances = []
    for lat_b, lon_b in zip(lat_series, lon_series):
        # Convert point B's coordinates from degrees to radians
        lat_b_rad = radians(lat_b)
        lon_b_rad = radians(lon_b)

        # Difference between the latitudes and longitudes
        d_lat = lat_b_rad - lat_a_rad
        d_lon = lon_b_rad - lon_a_rad

        # Haversine formula
        a = sin(d_lat / 2)**2 + cos(lat_a_rad) * cos(lat_b_rad) * sin(d_lon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = round((radius * c)*1000, 2)

        distances.append(distance)

    return distances

# Coordinates of point A
lat_a = 32.610982
lon_a = -38.706256

# Coordinates of the series of points


distances = calculate_distance(lat_a, lon_a, lad_list, lon_list)
print(distances)


In [5]:
# extract the world port index data
wpi = pd.read_csv('Wpi Data.csv')
wpi.head()

,World_port_index_number,Region_index,Main_port_name,Wpi_country_code,Latitude_degrees,Latitude_minutes,Latitude_hemisphere,Longitude_degrees,Longitude_minutes,Longitude_hemisphere,...,Services_elect_repair,Supplies_provisions,Supplies_water,Supplies_fuel_oil,Supplies_diesel_oil,Supplies_deck,Supplies_engine,Repair_code,Drydock,Railway
0,70,60,KEFLAVIK,IS,64,0,N,22,33,W,...,NaN,Y,Y,Y,Y,Y,Y,C,NaN,S
1,75,60,STRAUMSVIK,IS,64,3,N,22,3,W,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,80,60,HAFNARFJORDUR,IS,64,4,N,21,57,W,...,Y,Y,Y,Y,Y,Y,Y,B,NaN,M
3,90,60,SKERJAFJORDUR,IS,64,9,N,22,1,W,...,NaN,Y,Y,Y,Y,Y,Y,C,NaN,NaN
4,100,60,REYKJAVIK,IS,64,9,N,21,56,W,...,Y,Y,Y,Y,Y,Y,Y,B,L,L


In [12]:
# extract columns of interest
col_needed = ['Main_port_name', 'Wpi_country_code','Latitude_degrees','Longitude_degrees','Supplies_water', 'Supplies_fuel_oil', 'Supplies_diesel_oil']
wpi = wpi[col_needed]
wpi['Distance_distress_meters'] = distances
wpi.to_csv('Distance_Distress.csv', index = False)
wpi.head()

,Main_port_name,Wpi_country_code,Latitude_degrees,Longitude_degrees,Supplies_water,Supplies_fuel_oil,Supplies_diesel_oil,Distance_distress_meters
0,KEFLAVIK,IS,64,22,Y,Y,Y,3664790.54
1,STRAUMSVIK,IS,64,22,NaN,NaN,NaN,3680421.57
2,HAFNARFJORDUR,IS,64,21,Y,Y,Y,3684238.79
3,SKERJAFJORDUR,IS,64,22,Y,Y,Y,3691111.55
4,REYKJAVIK,IS,64,21,Y,Y,Y,3692900.69


In [13]:
# esterblish a connection fo postgress server
import psycopg2
def get_db_connection(host, database, username, password):
    conn = psycopg2.connect(
        host=host,
        database=database,
        user=username,
        password=password
    )
    
    return conn

# initialize connection function parameters
host = "localhost"
database = "World_Port_index"
username = "postgres"
password = "********"

# establish a connect
con = get_db_connection(host, database, username, password)
# create cursor
curr = con.cursor()

In [49]:
# create a table that will store data for nearest port with provisions in postgres server.
create_statement = 'CREATE TABLE distance_distress(Main_port_name VARCHAR, Wpi_country_code VARCHAR, Latitude_degrees NUMERIC, Longitude_degrees NUMERIC, Supplies_water VARCHAR, Supplies_fuel_oil VARCHAR, Supplies_diesel_oil VARCHAR, Distance_distress_meters NUMERIC)'
curr.execute(create_statement)
con.commit()

In [50]:
# copy data to table
with open('Distance_Distress.csv', 'r') as f:
    next(f)
    curr.copy_from(f, 'distance_distress', sep=',')
    
# commit connection   
con.commit()

In [14]:
sql = """select main_port_name, 
	wpi_country_code, 
	latitude_degrees, 
	longitude_degrees
from distance_distress
where supplies_water = 'Y' and supplies_fuel_oil = 'Y' and supplies_diesel_oil = 'Y'
order by distance_distress_meters 
limit 5 """

pd.read_sql(sql, con)

,main_port_name,wpi_country_code,latitude_degrees,longitude_degrees
0,HORTA,PT,38.0,28.0
1,ANGRA DO HEROISMO,PT,38.0,27.0
2,PRAIA DE VITORIA,PT,38.0,27.0
3,PONTA DELGADA,PT,37.0,25.0
4,FUNCHAL,PT,32.0,16.0


In [ ]:
#close connection.
curr.close()
con.close()